# Project 3: Classification with Logistic Regression and SVM

Before we start, please put your name and CUID in following format

: Firstname LASTNAME, #00000000   //   e.g. Nianyi LI, #12345678

**Your Answer:**   
Your NAME, #XXXXXXXX

# General Rules of the Project Submission

Python 3 and [Matplotlib](https://matplotlib.org/) will be used throughout the semseter, so it is important to be familiar with them. It is strongly suggested to go through [Stanford CS231n](http://cs231n.github.io/python-numpy-tutorial/) and [CS228](https://github.com/kuleshov/cs228-material/blob/master/tutorials/python/cs228-python-tutorial.ipynb) for more detailed Python and numpy tutorials if you haven't had used Python before. 

In some cells and files you will see code blocks that look like this:

```python
##############################################################################
#                    TODO: Write the equation for a line                     #
##############################################################################
pass
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################
```

You should replace the `pass` statement with your own code and leave the blocks intact, like this:

```python
##############################################################################
#                    TODO: Write the equation for a line                     #
##############################################################################
y = m * x + b
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################
```

When completing the notebook, please adhere to the following rules:
- Do not write or modify any code outside of code blocks
- Follow the instruction of the project description carefully
- Run all cells before submitting. <span style="color:red">**You will only get credit for code that has been run!**.</span>

The last point is extremely important and bears repeating:

### We will not re-run your notebook -- <span style="color:red">you will only get credit for cells that have been run</span>

### File name
Your Python program should be named **yourlastname_yourfirstname_P3.ipynb**, then zip it and upload to Canvas

# Project Description

For this project we will apply both **Logistic Regression** and **SVM** to predict whether capacitors from a fabrication plant pass quality control based (QC) on two different tests. To train your system and determine its reliability you have a set of 118 examples. The plot of these examples is show below where a red x is a capacitor that failed QC and the green circles represent capacitors that passed QC.

<div>
<img src="https://nianyil.people.clemson.edu/CPSC_4430/P3_new.png" width="500"/>
</div>


## Data File

Two text files with the data is available on Canvas: a training set of 85 examples and a test set of 33 examples. Both are formatted as
- First line: **m** and **n**, tab separated
- Each line after that has two real numbers representing the results of the two tests, followed by a *1.0* if the capacitor *passed* QC and a *0.0* if it *failed* QC—tab separated.

You need to write a code to read data from the file. You **can** use packages, such as **panda**, to load the data.


In [15]:
##############################################################################
#         TODO: Write the code for reading data from file                    #
##############################################################################
import numpy as np
from tabulate import tabulate
x1, x2, y = np.loadtxt('P3train.txt', skiprows = 1, unpack = True)
t_x1, t_x2, t_y = np.loadtxt('P3test.txt', skiprows = 1, unpack = True)
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################

In [17]:
x = np.column_stack([x1,x2])
X = np.matrix(x)
print(X)

[[ 0.13191   -0.51389  ]
 [ 0.29896    0.61915  ]
 [ 0.13767    0.57529  ]
 [ 0.82316    0.27558  ]
 [ 0.59274   -0.7405   ]
 [ 0.50634    0.75804  ]
 [-0.13306   -0.4481   ]
 [-0.4038     0.70687  ]
 [-0.74366   -0.25804  ]
 [-0.13882    0.54605  ]
 [-0.092742   0.68494  ]
 [-0.17339    0.64839  ]
 [-0.29435    0.77997  ]
 [-0.60541    0.59722  ]
 [-0.59965   -0.41886  ]
 [ 0.63882    0.88962  ]
 [-0.375      0.50219  ]
 [ 0.28744   -0.76974  ]
 [-0.69758    0.68494  ]
 [ 0.73675   -0.18494  ]
 [-0.72062    0.53874  ]
 [-0.50749    0.90424  ]
 [ 0.60426    0.59722  ]
 [-0.28859   -0.060673 ]
 [ 0.22408    0.77997  ]
 [ 0.38537   -0.56506  ]
 [ 0.54666    0.48757  ]
 [ 0.22984   -0.41155  ]
 [ 0.2932    -0.2288   ]
 [-0.092742   0.55336  ]
 [-0.62846    0.33406  ]
 [ 0.76555    0.50219  ]
 [ 0.86348   -0.082602 ]
 [-0.28283    0.47295  ]
 [-0.52477    0.2098   ]
 [-0.39804    0.034357 ]
 [-0.43836    0.21711  ]
 [-0.54781    0.70687  ]
 [ 0.92684    0.3633   ]
 [-0.49021   -0.3019   ]


Your assignment is to use what you have learned from the class slides and homework to create (**from scratch in Python**, not by using Logistic Regression library function!) a **Logistic Regression** and **SVM** binary classifier to predict whether each capacitor in the test set will pass QC. 

## Logistic Regression

You are free to use any model variation and any testing or training approach we have discussed for logistic regression. In particular, since this data is not linear, I assume you will want to add new features based on power of the original two features to create a good decision boundary. $w_0 + w_1x_1 + w_2x_2$ is not going to work!
One choice might be
- $\textbf{w}^T \textbf{x} = w_0 + w_1x_1 + w_2x_2 + w_3x_3 + w_4x_4 + w_5x_5 +w_6x_6 + w_7x_7 + w_8x_8$    where the new features are created as follows:

| New Features |From Original Features |
| --- | --- |
|$x_1$	| $x_1$|
|$x_2$	| $x_1^2$|
|$x_3$	| $x_2$||
|$x_4$	| $x_1x_2$|
|$x_5$	| $x_1x_2^2$|
|$x_6$	| $x_2^2$|
|$x_7$	| $x_1^2x_2$|
|$x_8$	| $x_1^2x_2^2$|

Note that it is easy to create a small Python program that reads in your  original features, uses a nested loop to create the new features and then writes them to a file:

```python
thePower = 2
for j in range(thePower+1): 
    for i in range(thePower+1):
        temp = (x1**i)*(x2**j)
        if (temp != 1):
            fout1.write(str(temp)+"\t") fout1.write(str(y)+"\n")
```

With a few additions to the code, you can make a program to create combinations of any powers of $x_1$ and $x_2$!

In [13]:
##############################################################################
#           TODO: Define the Logistic regression models                      #
##############################################################################
def logistic_model():
    e_z = np.exp(-(compute_wTx(x1, x2, w)))
    g_z = 1 / (1 + (e_z))
    return g_z
def compute_wTx(x1, x2, w):
    wTx = (
        w[0] + 
        w[1] * x1 + 
        w[2] * x1**2 + 
        w[3] * x2 + 
        w[4] * x1 * x2 + 
        w[5] * x1 * x2**2 + 
        w[6] * x2**2 + 
        w[7] * x2 * x1**2 + 
        w[8] * (x1**2) * (x2**2)
    )
    return wTx
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################

## Optimization using Gradient Decent

Once you have defined the logistic regression model, you need to find the weights using the Gradient Decent algorithm. You need to implement the Vanilla Gradient Decent from scratch in Python.

You need to specify the hyperparameters of GD, and plot the training loss curve (**J-curve**). The loss function should be the binary cross-entropy loss function that we introduced.

In [14]:
##############################################################################
#           TODO: Implement the Gradient Decent Algorithm                    #
##############################################################################
# Define the hyperparameters:
# Numbers of epoch (epoch_num), learning rate (lr), and the initial weights(w)
epoch_num = pass
lr = pass
w = pass
J = pass

# Define the loss:
def cross_entropy_loss(y_pred,y):
    m = len(y)
    J = -(1/m) * np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
    return J

# Calculate the gradient function:
def gradient_func(J,w):
    
    return gradient_value
    
# Implement the Gradient decent algorithm using for loop
def Vanilla_GD(epoch_num,lr,w,J):
    for epoch in range(epoch_num):
        x = np.column_stack([x1,x2])
        X = np.matrix(x)
        # Make predictions
        z = np.dot(X, w)
        y_pred = sigmoid(z)
        
        # Calculate the gradient
        gradient = gradient_func(J,w)
        
        # Update the weights
        w = w - lr * gradient           
    return w
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################

SyntaxError: invalid syntax (4250092216.py, line 6)

Next, print out the final weights and plot the **J-curve/Loss curve** of training. 

In [ ]:
##############################################################################
#                     TODO: Plot the J curve                                 #
##############################################################################
# Print out the final weights
print(w)

# Plot the J curve w.r.t. the iteration numbers
plt.plot(range(epoch_num), cost)
plt.xlabel('Iteration')
plt.ylabel('Cost')
plt.title('J-Curve / Loss Curve of Training')
plt.show()
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################

Based on your data and plot, you should then briefly discuss how you can ensure that the model is well trained.

**Your Answer:**  

## Model Evaluation

Evaluate the performance on testing set:
- Print out the confusion matrix
- Calculate and print out the *accuracy*, *precision*, *recall*, and *F1* value of your model

**Note that:**
- For **undergrads** *(CPSC 4430)* the final accuracy of both algorithms on your test set should be higher than  <span style="color:red">**70%**</span>
- For **graduate-level** *(CPSC 6430)* the final accuracy of both algorithms on your test set should be higher than  <span style="color:red">**85%**</span>


In [ ]:
##############################################################################
#                           TODO: Model Evaluation                           #
##############################################################################
pass
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################

## Support Vector Machine (SVM)

In this part, you need to use the previous training and testing data file. 

You are **allowed** to use the svm functions in the **Scikit-learn** library and don’t need to implement the algorithm from scratch.

- You need to try at least **three** different kernel functions of SVM, and pick the **best** model.
- You need to print out the final weights got from your best SVM model.

**Note that:**
- For **undergrads** *(CPSC 4430)* the final accuracy of both algorithms on your test set should be higher than  <span style="color:red">**70%**</span>
- For **graduate-level** *(CPSC 6430)* the final accuracy of both algorithms on your test set should be higher than  <span style="color:red">**85%**</span>

In [ ]:
##############################################################################
#                      TODO: Classfication using SVM                         #
##############################################################################
# Pick the best model
pass

# Print out the final weights
pass
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################

## Visualize Decision Boundary and Model Comparision

You need to plot the decision boundary of Logistic Regression and SVM that you previously trained separately. 

In [ ]:
##############################################################################
#                   TODO: Plot the Decision Boundary                         #
##############################################################################
pass
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################

Based on your data and plot, you should then briefly discuss which one has better performance and why.

**Your Answer:**  